<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/PrescriptionFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
hemo = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Hemodialysis.csv', encoding='latin1')
lab = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Lab.csv')
prescription = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Prescription.csv')

In [3]:
# Create a list from hemo['PatientUID']
patient_uid_list = hemo['PatientUID'].unique().tolist()
prescription = prescription[~prescription['PatientUID'].isin(patient_uid_list)]
lab = lab[~lab['PatientUID'].isin(patient_uid_list)]

#  **📁Presecription**

In [4]:
df = pd.DataFrame(prescription)
df

,PatientUID,PatientVisitUID,DispensedDttm,DrugName,GenericName,Quantity,DrugFrequency,Dosage,DosageForm,DrugDuration
0,2299,2024049,2558-02-19 14:48:15.693,Folic Acid 5 mg TAB,Folivit tab 5 mg,90.0,od pc |วันละ 1 ครั้ง หลังอาหารเช้า,1.0,1 TAB,90.0
1,2299,2024049,2558-02-19 14:48:15.693,Vitamin B Complex TAB,Vitamin B Complex TAB,180.0,bid pc |วันละ 2 ครั้ง หลังอาหารเช้า เย็น,1.0,1 TAB,90.0
2,2299,2024049,2558-02-19 14:48:15.693,losartan(Loranta) 50 mg TAB,Loranta film-coated tab 50 mg,90.0,od pc |วันละ 1 ครั้ง หลังอาหารเช้า,1.0,1 TAB,90.0
3,2299,2024049,2558-02-19 14:48:15.693,lorazepam(Anxira) 0.5 mg TAB,Anxira tab 0.5 mg,30.0,hs |วันละ 1 ครั้ง ก่อนนอน,1.0,1 TAB,30.0
4,2299,2024049,2558-02-19 14:47:55.953,ChalkTAB-1.5 TAB,Chalktab 1500 mg,90.0,od pc |วันละ 1 ครั้ง หลังอาหารเช้า,1.0,1 TAB,90.0
...,...,...,...,...,...,...,...,...,...,...
9530447,3437526,10191122,2565-12-27 03:00:00.563,Infusion plabottle set (IV.set),NaN,2.0,NaN,NaN,NaN,1.0
9530448,3437526,10191122,2565-12-27 03:00:00.563,Normal Saline 0.9% IV 1000 ml,Normal Saline 0.9% IV 1000 ml,1.0,As Direct |ใช้ตามแพทย์สั่ง,NaN,0 ml,1.0
9530449,3437526,10191122,2565-12-27 03:00:00.563,Normal Saline 0.9% IV 100 ml,Normal Saline 0.9% IV 100 ml,2.0,As Direct |ใช้ตามแพทย์สั่ง,NaN,0 ml,1.0
9530450,3437526,10191122,NaN,Minimun Volume Extension with T connector,NaN,1.0,NaN,NaN,NaN,1.0


## 📖 **List Description**
```
- **drugCol1 / genericCol1** == list of lower case
- **drugCol2 / genericCol2** == store value not start with digits
- **drugCol3 / genericCol3** == clean unit from lab.csv
- **drugCol4 / genericCol4** == clean words to remove from wordcloud
```

In [5]:
import pandas as pd
import re

# Lowercase and store values from 'DrugName' column
drugCol1 = [str(i).lower() for i in df['DrugName']]

# Lowercase and store values from 'GenericName' column
genericCol1 = [str(i).lower() for i in df['GenericName']]

# Store drug names
drugCol2 = []
genericCol2 = []

# Process 'DrugName' column
for value in drugCol1:
    if isinstance(value, str):
        if value[0].isdigit():
            drugCol2.append(value)
        else:
            valList = value.split()
            index_begin_with_digit = len(valList)
            for i in range(len(valList)):
                if valList[i][0].isdigit():
                    index_begin_with_digit = i
                    break
            newval = " ".join(valList[:index_begin_with_digit])
            drugCol2.append(newval)

# Process 'GenericName' column
for value in genericCol1:
    if isinstance(value, str):
        if value[0].isdigit():
            genericCol2.append(value)
        else:
            valList = value.split()
            index_begin_with_digit = len(valList)
            for i in range(len(valList)):
                if valList[i][0].isdigit():
                    index_begin_with_digit = i
                    break
            newval = " ".join(valList[:index_begin_with_digit])
            genericCol2.append(newval)


## 🥼 **Remove Unit words from lab.csv**

In [6]:
# Define the unit_unique_list containing words to be removed
unit_unique_list = lab['Unit'].unique().tolist()

# Function to remove words from a list of strings
def remove_words_from_list(input_list, words_to_remove):
    result_list = []
    for item in input_list:
        # Check if the item is a string or convert to string if it's not
        if not isinstance(item, str):
            item = str(item)

        # Split the item into words and remove the words in words_to_remove
        words = item.split()
        words = [word for word in words if word not in words_to_remove]
        result_list.append(" ".join(words))
    return result_list

# Process 'drugCol2' and 'genericCol2'
drugCol3 = remove_words_from_list(drugCol2, unit_unique_list)
genericCol3 = remove_words_from_list(genericCol2, unit_unique_list)

In [7]:
words_to_remove = ['ed.tab','m)','nan','inj', 'cap', 'mg', 'ml', 'tab',
                   '[7402]', '[5702]', '[5703]', '[0604]', '[5606]', '[024840]',
                   '[napha]', '[5704]', '(ฝาก)', '(เลิกใช้)','ch.22','[024840]',
                   '[0604]','[5606]','[5702]','[5703]''[5704]','[7402]','tab(ฝาก)']

In [8]:
drugCol4 = []
for i in drugCol3:
    for word in words_to_remove:
        i = i.replace(word, "")
    i = re.sub(r'\bno\.\d+(\.\d+)?\b', '', i)
    drugCol4.append(i)

In [9]:
genericCol4 = []
for i in genericCol3:
    for word in words_to_remove:
        i = i.replace(word, "")
    i = re.sub(r'\bno\.\d+(\.\d+)?\b', '', i)
    genericCol4.append(i)

In [39]:
import pandas as pd
import re
from itertools import zip_longest

# Assuming you have already processed and filled the lists drugCol2, genericCol2, drugdigits, and genericdigits.

# Combine drugCol4 and genericCol4 into DataFrame table1
table1 = pd.DataFrame({'DrugName': drugCol4, 'GenericName': genericCol4})


In [41]:
table1

,DrugName,GenericName
0,folic acid,folivit
1,vitamin b complex,vitamin b complex
2,losartan(loranta),loranta film-coated
3,lorazepam(anxira),anxira
4,chalk-1.5,chalk
...,...,...
7845034,infusion plabottle set (iv.set),
7845035,normal saline,normal saline
7845036,normal saline,normal saline
7845037,minimun volume extension with t connector,


## 🗓 **Dealing with Drug Start with digits**

In [42]:
a = table1[table1['DrugName'].str.match(r'^\d') | table1['GenericName'].str.match(r'^\d')]
a

,DrugName,GenericName
50195,2% chlorhexidine in 70% alcohol 450,2% chlorhexidine in 70% alcohol 500
50206,2% chlorhexidine in 70% alcohol 450,2% chlorhexidine in 70% alcohol 500
50260,3tc lamivudine 150,lamivudine mylan film-coated
50263,3tc lamivudine 150,lamivudine mylan film-coated
51087,3tc 150,3tc 150
...,...,...
7689649,3tc 300,lamivir
7689660,3tc 300,lamivir
7689667,3tc 300,lamivir
7693783,5-fu 10/0.2 . for subconjunctival,5-fu 10/0.2 . for subconjunctival


In [45]:
import pandas as pd

# Function to clean and group by keywords
def clean_and_group(item):
    if item.startswith("2% chlorhexidine in 70% alcohol"):
        return "2% chlorhexidine in 70% alcohol"
    elif item.startswith("5-fu"):
        return "5-fu"
    elif item.startswith("3tc lamivudine"):
        return "3tc lamivudine"
    elif item.startswith("3tc"):
        return "3tc"
    elif item == "Md Personnel Checkup stool examination":
        return "Stool"
    else:
        return item

# Modify the 'RequestItemName' column directly
table1['DrugName'] = table1['DrugName'].apply(clean_and_group)
table1['GenericName'] = table1['GenericName'].str.extract(r'^(.*?)( \(|$)')[0].str.strip()

In [46]:
b = table1[table1['DrugName'].str.match(r'^\d') | table1['GenericName'].str.match(r'^\d')]
b['DrugName'].unique()

array(['2% chlorhexidine in 70% alcohol', '3tc lamivudine', '3tc', '5-fu',
       'sur-fit drainable po'], dtype=object)

In [47]:
table1

,DrugName,GenericName
0,folic acid,folivit
1,vitamin b complex,vitamin b complex
2,losartan(loranta),loranta film-coated
3,lorazepam(anxira),anxira
4,chalk-1.5,chalk
...,...,...
7845034,infusion plabottle set (iv.set),
7845035,normal saline,normal saline
7845036,normal saline,normal saline
7845037,minimun volume extension with t connector,


## 🇹🇭 **Thai Value**

In [48]:
# Define a function to check if a string contains Thai characters
def has_thai_characters(text):
    return any("\u0E00" <= char <= "\u0E7F" for char in text)

# Filter rows where DrugName or GenericName contain Thai characters
th = table1[table1['DrugName'].apply(has_thai_characters) | table1['GenericName'].apply(has_thai_characters)]
th

,DrugName,GenericName
33,เฝือกพยุงระดับเอว l - s support no. l,
220,intravenous disposable catheter ch.20 ความยาวป...,
226,แผ่นสำหรับตรวจการติดเชื้อเฮลิโคแบคเตอร์ ไพโลไร,
236,transparent dressing ขนาด,
363,intravenous disposable catheter ความยาวประมาณ,
...,...,...
7845001,แผ่นสำหรับตรวจการติดเชื้อเฮลิโคแบคเตอร์ ไพโลไร,
7845010,ฝาปิดจุก three way sterile,
7845011,ชุดให้อาหารทางสายยาง ขนาด,
7845018,transparent dressing ขนาด,


#### **Case1**
    Pattern Eng end with Thai

In [50]:

# Define a pattern to match English words followed by parentheses with Thai content
english_pattern = r'^[a-zA-Z\s]+\s*\([^()]*[\u0E00-\u0E7F]+[\s\S]*\)$'

# Function to remove Thai words and keep only English words
def remove_thai(text):
    return re.sub(english_pattern, '', text)

# Apply the function to 'DrugName' and 'GenericName' columns
table1['DrugName'] = table1['DrugName'].apply(remove_thai)
table1['GenericName'] = table1['GenericName'].apply(remove_thai)


In [51]:
table1

,DrugName,GenericName
0,folic acid,folivit
1,vitamin b complex,vitamin b complex
2,losartan(loranta),loranta film-coated
3,lorazepam(anxira),anxira
4,chalk-1.5,chalk
...,...,...
7845034,infusion plabottle set (iv.set),
7845035,normal saline,normal saline
7845036,normal saline,normal saline
7845037,minimun volume extension with t connector,


In [52]:
# Filter rows where DrugName or GenericName contain Thai characters
t = table1[table1['DrugName'].apply(has_thai_characters) | table1['GenericName'].apply(has_thai_characters)]
t['DrugName'].nunique()

1987

In [57]:
t

,DrugName,GenericName
33,เฝือกพยุงระดับเอว l - s support no. l,
220,intravenous disposable catheter ch.20 ความยาวป...,
226,แผ่นสำหรับตรวจการติดเชื้อเฮลิโคแบคเตอร์ ไพโลไร,
236,transparent dressing ขนาด,
363,intravenous disposable catheter ความยาวประมาณ,
...,...,...
7845001,แผ่นสำหรับตรวจการติดเชื้อเฮลิโคแบคเตอร์ ไพโลไร,
7845010,ฝาปิดจุก three way sterile,
7845011,ชุดให้อาหารทางสายยาง ขนาด,
7845018,transparent dressing ขนาด,


#### **Case 2**
    Thai words end with Eng + digits

In [53]:
# Define a pattern to match Thai text followed by English words or digits
pattern = r'[\u0E00-\u0E7F]+[a-zA-Z0-9\s]*$'

# Function to extract Thai text
def extract_thai(text):
    match = re.search(pattern, text)
    return match.group() if match else ''

# Apply the function to 'DrugName' and 'GenericName' columns
table1['DrugName'] = table1['DrugName'].apply(extract_thai)
table1['GenericName'] = table1['GenericName'].apply(extract_thai)


In [54]:
t2 = table1[table1['DrugName'].apply(has_thai_characters) | table1['GenericName'].apply(has_thai_characters)]
t2['DrugName'].nunique()

784